In [ ]:
from pathlib import Path, PurePath
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from analysis import read_behaviors, read_moea_fitness, unpickle_moea
from utility.util_functions import calc_str_disparity, euclidean_distance
from utility.visualize import plot_path, plot_moea_fitness, plot_surrogate_model, plot_str_disparity
import pickle
import numpy as np
from deap import base, creator
from evolution.simulation import Disparity
init_notebook_mode(connected=True)


In [ ]:
# get the path for all behavioral features
files = sorted(Path('data/results/').glob('transferability_*/behavioral_features.dat'))
fitness_f = sorted(Path('data/results/').glob('transferability_*/fitness.dat'))

In [ ]:
columns = [
        'avg_left', 'avg_right',
        's1', 's2', 's3', 's4', 's5', 's6', 's7',
        'area0_percentage',
        'area1_percentage',
        'area2_percentage',
    ]

In [ ]:
fitness_f

In [ ]:
behaviors = read_behaviors([files[3]])[0]

In [ ]:
behaviors.loc[behaviors['genome_id'] == 230].head()

In [ ]:
behaviors.describe()

In [ ]:
b_grouped = behaviors.groupby(['genome_id']).filter(lambda x: len(x) > 1).groupby(['genome_id', 'simulation']).first()

In [ ]:
b_grouped.head()

In [ ]:
b_grouped.index.get_values()

In [ ]:
euclidean_distance(b_grouped[columns].iloc[1], b_grouped[columns].iloc[0])

In [ ]:
for i in range(1, len(b_grouped), 2):
    print('genome_id:', 
          b_grouped[columns].iloc[i].name[0],
          'actual diversity',
          euclidean_distance(
              b_grouped[columns].iloc[i],
              b_grouped[columns].iloc[i-1]
          )
    )

In [ ]:
fitness_data = read_moea_fitness([fitness_f[3]])[0]

In [ ]:
fitness_data.head()

In [ ]:
hof = unpickle_moea('data/results/transferability_simulation_13/hof')

In [ ]:
len(hof)

In [ ]:
plot_moea_fitness(fitness_data, hof)

In [ ]:
for ind in hof:
    print(ind.key, ind.fitness.values)

In [ ]:
genomes_file = sorted(Path('data/results/transferability_simulation_13/deap_inds').glob('*_genome.pkl'))
transfered_file = sorted(Path('data/results/transferability_simulation_13/deap_inds').glob('*_transformed_genome_.pkl'))
genomes = [unpickle_moea(genome) for genome in genomes_file]
transferred = [unpickle_moea(genome) for genome in transfered_file]

In [ ]:
transfered_file

In [ ]:
len(transfered_file)

In [ ]:
g = sorted(genomes, key=lambda x: x.key)

In [ ]:
t = sorted(transferred, key=lambda x: x.key)

In [ ]:
sim_genome = list(filter(lambda x: x.key == 171, g))[0]

In [ ]:
print(sim_genome.key, sim_genome.fitness.values)

In [ ]:
trans_genome = list(filter(lambda x: x.key == 171, t))[0]

In [ ]:
print(trans_genome.key, trans_genome.fitness.values)

In [ ]:
trans_genome.sim_time

In [ ]:
plot_path([sim_genome, trans_genome])

In [ ]:
calc_str_disparity(trans_genome.position, sim_genome.position)

In [ ]:
plot_surrogate_model(fitness_data)

In [ ]:
# merge  
union_t = [(x, y) for x in g for y in t if x.key == y.key]
for s, t in union_t:
    np.array_equal(s, t)
    assert s.key == t.key

In [ ]:
str_disparity = Disparity()

In [ ]:
for s, t in union_t:
    print(s.gen, end=" ")

In [ ]:
str_disparities = [(s.gen, s.key, s.fitness.values[0], s.fitness.values[1], calc_str_disparity(t.position, s.position)) for s, t in union_t]    

In [ ]:
str_disparities

In [ ]:
np.array([str_disparity[1] for str_disparity in str_disparities])
np.array([real_disparity[2] for real_disparity in str_disparities])

filter_disparities = list(filter(lambda x: x[2] < 5, str_disparities))
filter_disparities[:4]

In [ ]:
plot_str_disparity(filter_disparities)

In [ ]:
np.arange(0, 11)